# Eflux2 applied to *A. niger*

In [1]:
import sys
import pandas as pd
import cobra
sys.path.append('../../src')
from eflux2 import EFlux2

## Read global proteomics normalized to the reference strain SF ABF180_17_R2

[(βAl-3HP)++, pyc2++, Δald6a+NeoR] (Strain ABF-10216R8, ICE ID ABF_011245)

In [2]:
global_prot = pd.read_csv('../../data/round2/normalized_global_proteomics.csv', index_col='Transcript')
global_prot

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
Transcript,,,,,,,,,,,,,,,,,,,,,
1170085,0.967266,0.996633,0.992893,0.988900,1.005546,0.994246,0.995436,0.984235,0.985065,0.987202,...,0.996191,1.000241,0.981752,1.003174,1.000581,1.006389,0.998983,0.974427,0.986005,0.982189
1141495,1.012991,1.010041,1.011290,1.012467,1.008045,1.010177,1.032464,1.009299,1.012114,1.014553,...,1.010923,1.019918,1.003567,1.020978,1.010532,1.022859,1.026123,1.015454,1.016219,1.003530
201546,0.979618,0.980405,0.973220,0.981140,0.998126,0.998079,0.991908,0.981119,0.990187,0.990484,...,0.988765,0.994606,0.979760,0.993260,1.001902,0.996684,0.994010,0.982122,0.986781,0.980817
1147898,0.995988,1.000736,0.999673,1.021090,1.020466,1.023879,1.007351,1.005843,1.003600,1.006999,...,1.007180,1.001447,1.008006,1.004574,1.003261,1.014986,1.022990,1.002505,0.996382,1.002931
1142869,1.018378,1.017542,1.017273,1.015201,1.009343,1.003975,1.016892,1.010228,1.026211,1.010030,...,1.000423,1.002124,1.005939,1.004112,1.001554,1.010091,0.992440,1.000478,0.998578,0.997856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117615,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
210213,1.000000,1.000000,1.000000,1.000000,1.000000,0.926492,0.928163,1.000000,0.951025,1.000000,...,1.013350,1.024441,1.026118,1.015304,1.012428,0.997734,0.990888,0.970595,1.000000,0.954842
1119048,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Read iJB1325 with non-native genes added for 3HP production

In [3]:
# todo: update gurobi license (not required to use gurobi)
model = cobra.io.load_json_model('../../models/iJB1325_HP.json')
#model.solver = 'gurobi'
#model.solver

In [22]:
model.medium

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

In [24]:
model.reactions.BOUNDARY_HNO3e.lower_bound = 0.0
model.reactions.BOUNDARY_NH4e.lower_bound = -1000.0

In [25]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
GLCe,BOUNDARY_GLCe,10,6,100.00%
NH4e,BOUNDARY_NH4e,4.307,0,0.00%
O2e,BOUNDARY_O2e,10.73,0,0.00%
PIe,BOUNDARY_PIe,0.1275,0,0.00%
SLFe,BOUNDARY_SLFe,0.05042,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CO2e,BOUNDARY_CO2e,-13.87,1,100.00%
H2Oe,BOUNDARY_H2Oe,-27.03,0,0.00%
He,BOUNDARY_He,-4.32,0,0.00%
BIOMASS,DRAIN_Biomass,-1.042,0,0.00%


## Read line rates

In [5]:
line_rates = pd.read_csv('../../data/round2/Line_rates.csv', index_col='Line Name')
line_rates.head()

,Omics Sample ID,Strain (ICE),Genotype,GLCe (mmol/gDW/hr),3hpp_e (mmol/gDW/hr),ETHe (mmol/gDW/hr),EOLe (mmol/gDW/hr),CITe (mmol/gDW/hr)
Line Name,,,,,,,,
SF ABF180_1_R1,SF ABF180_1_R1,ABF_008340,wild-type,-3.824936,0.000000,-0.792609,0.003407,0.013740
SF ABF180_1_R2,SF ABF180_1_R2,ABF_008340,wild-type,-3.557249,0.000000,-0.678005,0.012109,0.009979
SF ABF180_1_R3,SF ABF180_1_R3,ABF_008340,wild-type,-3.415471,0.000000,-1.080524,0.010234,0.009554
SF ABF180_2_R1,SF ABF180_2_R1,ABF_008343,(βAl-3HP)+,-5.928543,2.040397,-0.352736,0.022628,-0.001312
SF ABF180_2_R2,SF ABF180_2_R2,ABF_008343,(βAl-3HP)+,-5.792077,2.099299,-0.539260,0.005308,-0.002935


## Normalize uptake and secretion rates by glucose uptake rate of reference study line.

In [6]:
ref_line = 'SF ABF180_17_R2'

ref_line_glucose_rate = -line_rates.loc[ref_line,'GLCe (mmol/gDW/hr)']
ref_line_glucose_rate

6.277127634711532

In [7]:
normalized_line_rates = line_rates.drop(['Omics Sample ID', 'Strain (ICE)', 'Genotype'], axis=1)
normalized_line_rates = normalized_line_rates.divide(ref_line_glucose_rate, axis=1)
normalized_line_rates

,GLCe (mmol/gDW/hr),3hpp_e (mmol/gDW/hr),ETHe (mmol/gDW/hr),EOLe (mmol/gDW/hr),CITe (mmol/gDW/hr)
Line Name,,,,,
SF ABF180_1_R1,-0.609345,0.000000,-0.126269,0.000543,0.002189
SF ABF180_1_R2,-0.566700,0.000000,-0.108012,0.001929,0.001590
SF ABF180_1_R3,-0.544114,0.000000,-0.172137,0.001630,0.001522
SF ABF180_2_R1,-0.944467,0.325053,-0.056194,0.003605,-0.000209
SF ABF180_2_R2,-0.922727,0.334436,-0.085909,0.000846,-0.000467
...,...,...,...,...,...
SF ABF180_23_R2,-0.989133,0.478907,-0.121174,0.012959,0.001360
SF ABF180_23_R3,-1.137562,0.564224,-0.150257,0.013712,0.001255
SF ABF180_24_R1,-1.124896,0.949444,-0.043846,0.024140,0.002688


In [8]:
normalized_line_rates[(normalized_line_rates < 0).any(axis=1)]

,GLCe (mmol/gDW/hr),3hpp_e (mmol/gDW/hr),ETHe (mmol/gDW/hr),EOLe (mmol/gDW/hr),CITe (mmol/gDW/hr)
Line Name,,,,,
SF ABF180_1_R1,-0.609345,0.000000,-0.126269,0.000543,0.002189
SF ABF180_1_R2,-0.566700,0.000000,-0.108012,0.001929,0.001590
SF ABF180_1_R3,-0.544114,0.000000,-0.172137,0.001630,0.001522
SF ABF180_2_R1,-0.944467,0.325053,-0.056194,0.003605,-0.000209
SF ABF180_2_R2,-0.922727,0.334436,-0.085909,0.000846,-0.000467
...,...,...,...,...,...
SF ABF180_23_R2,-0.989133,0.478907,-0.121174,0.012959,0.001360
SF ABF180_23_R3,-1.137562,0.564224,-0.150257,0.013712,0.001255
SF ABF180_24_R1,-1.124896,0.949444,-0.043846,0.024140,0.002688


In [12]:
# Instead of replacing negative values with zero
# normalized_line_rates[normalized_line_rates < 0] = 0.0
# Use negative values for uptake and positive values for secretion

In [9]:
normalized_line_rates.to_csv('../../data/round2/normalized_line_rates.csv')

## Run Eflux

In [16]:
const_dict = {
       'BOUNDARY_GLCe': 'GLCe (mmol/gDW/hr)',
       'EX_3hpp_e': '3hpp_e (mmol/gDW/hr)',
       'BOUNDARY_ETHe': 'ETHe (mmol/gDW/hr)',
       'BOUNDARY_EOLe': 'EOLe (mmol/gDW/hr)',
       'BOUNDARY_CITe': 'CITe (mmol/gDW/hr)'
}

In [26]:
fluxes = {}
for rep in global_prot.columns:
    with model:
        for k, v in const_dict.items():
            model.reactions.get_by_id(k).lower_bound = normalized_line_rates[v][rep]
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")

SF ABF180_1_R1
FBA status optimal
FBA solution 0.030858699893925607
EFlux2 status optimal
EFlux2 solution 14.87049144842007
SF ABF180_1_R2
FBA status optimal
FBA solution 0.029578318688677066
EFlux2 status optimal
EFlux2 solution 14.130122065371996
SF ABF180_1_R3
FBA status optimal
FBA solution 0.028871452729212002
EFlux2 status optimal
EFlux2 solution 13.75166297674635
SF ABF180_2_R1
FBA status optimal
FBA solution 0.03421840475790549
EFlux2 status optimal
EFlux2 solution 29.06981438764832
SF ABF180_2_R2
FBA status optimal
FBA solution 0.0334386543094213
EFlux2 status optimal
EFlux2 solution 25.978376136324933
SF ABF180_2_R3
FBA status optimal
FBA solution 0.03490483912437181
EFlux2 status optimal
EFlux2 solution 34.92282563217087
SF ABF180_3_R1
FBA status optimal
FBA solution 0.03043473548067084
EFlux2 status optimal
EFlux2 solution 24.07110385121285
SF ABF180_3_R2
FBA status optimal
FBA solution 0.03169705755403263
EFlux2 status optimal
EFlux2 solution 27.096464009936785
SF ABF180_3

/Users/kimj972/anaconda3/envs/python3_cobrapy/lib/python3.6/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FBA status infeasible
FBA solution 0.06641065643871993
EFlux2 status infeasible
EFlux2 solution 48712.71395019922
SF ABF180_5_R2
FBA status infeasible
FBA solution 0.01504781506767918
EFlux2 status infeasible
EFlux2 solution 20.597180626247955
SF ABF180_5_R3
FBA status infeasible
FBA solution 0.1590199987540717
EFlux2 status infeasible
EFlux2 solution -8.81461222548947
SF ABF180_6_R1
FBA status optimal
FBA solution 0.024816750581155858
EFlux2 status optimal
EFlux2 solution 14.944023213887512
SF ABF180_6_R2
FBA status optimal
FBA solution 0.02906046841738322
EFlux2 status optimal
EFlux2 solution 18.59514235220953
SF ABF180_6_R3
FBA status optimal
FBA solution 0.029968964127822174
EFlux2 status optimal
EFlux2 solution 18.648008338736496
SF ABF180_7_R1
FBA status optimal
FBA solution 0.022504996931958683
EFlux2 status optimal
EFlux2 solution 18.701960210681314
SF ABF180_7_R2
FBA status optimal
FBA solution 0.023191954165814252
EFlux2 status optimal
EFlux2 solution 26.328487100757293
SF AB

In [27]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep, fluxes[rep].status)

SF ABF180_5_R1 infeasible
SF ABF180_5_R2 infeasible
SF ABF180_5_R3 infeasible
SF ABF180_17_R1 infeasible
SF ABF180_17_R2 infeasible
SF ABF180_17_R3 infeasible
SF ABF180_19_R1 infeasible
SF ABF180_19_R3 infeasible
SF ABF180_20_R1 infeasible
SF ABF180_20_R2 infeasible
SF ABF180_21_R1 infeasible
SF ABF180_21_R2 infeasible
SF ABF180_21_R3 infeasible
SF ABF180_22_R1 infeasible
SF ABF180_22_R2 infeasible
SF ABF180_22_R3 infeasible


### get eflux2 solutions by adding slack variables for infeasible cases

In [20]:
import numpy as np
from optlang.symbolics import add
from eflux2 import get_eflux2_bounds
from eflux.eflux2 import add_slack_variables_to_model

In [28]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep)
        # get eflux2 bounds from global proteomics
        eflux2_bounds = get_eflux2_bounds(model, global_prot[rep])
        upper_bounds = {k: v[1] if v[1] <= 1000.0 else 1000.0 for k, v in eflux2_bounds.items()}
        # build a relaxed model with slack variables
        relaxed_model = add_slack_variables_to_model(model, upper_bounds)
        # solve FBA to calculate the maximum CO2
        relaxed_model.tolerance = 1e-9
        fba_sol = relaxed_model.optimize()
        print('FBA status', fba_sol.status)
        print('FBA solution', fba_sol.objective_value)
        # Constrain the growth to the optimal value
        for r in relaxed_model.reactions:
            if r.objective_coefficient == 1.0:
                r.lower_bound = fba_sol[r.id]
        # minimize the sum of squared flux values
        relaxed_model.objective = relaxed_model.problem.Objective(add([r.flux_expression**2 for r in relaxed_model.reactions]), direction='min')
        eflux2_sol = relaxed_model.optimize()
        print('EFlux2 status', eflux2_sol.status)
        print('EFlux2 solution', eflux2_sol.objective_value)
        fluxes[rep] = eflux2_sol

SF ABF180_5_R1
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution 4.9093324742347827e-11
SF ABF180_5_R2
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution 4.736805118201147e-11
SF ABF180_5_R3
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -1.1967845666940655e-09
SF ABF180_17_R1
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -2.5123421961024682e-11
SF ABF180_17_R2
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -1.0824451831230847e-09
SF ABF180_17_R3
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -2.133435419890147e-09
SF ABF180_19_R1
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -1.2644389707123762e-10
SF ABF180_19_R3
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -5.020976105324271e-10
SF ABF180_20_R1
FBA status optimal
FBA solution 0.0
EFlux2 status optimal
EFlux2 solution -1.737

In [138]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep, fluxes[rep].status)

In [142]:
for rep in global_prot.columns:
    print(rep, round(fluxes[rep].objective_value,3))

SF ABF180_1_R1 14.87
SF ABF180_1_R2 14.13
SF ABF180_1_R3 13.752
SF ABF180_2_R1 29.07
SF ABF180_2_R2 25.978
SF ABF180_2_R3 34.923
SF ABF180_3_R1 24.071
SF ABF180_3_R2 27.096
SF ABF180_3_R3 19.801
SF ABF180_4_R1 38.638
SF ABF180_4_R2 24.266
SF ABF180_4_R3 25.665
SF ABF180_5_R1 0.0
SF ABF180_5_R2 0.0
SF ABF180_5_R3 -0.0
SF ABF180_6_R1 14.944
SF ABF180_6_R2 18.595
SF ABF180_6_R3 18.648
SF ABF180_7_R1 18.702
SF ABF180_7_R2 26.328
SF ABF180_7_R3 22.683
SF ABF180_8_R1 20.314
SF ABF180_8_R2 16.595
SF ABF180_8_R3 25.375
SF ABF180_9_R1 32.711
SF ABF180_9_R2 18.063
SF ABF180_9_R3 29.124
SF ABF180_10_R1 20.818
SF ABF180_10_R2 36.134
SF ABF180_10_R3 13.093
SF ABF180_11_R1 29.995
SF ABF180_11_R2 52.29
SF ABF180_11_R3 39.671
SF ABF180_12_R1 51.043
SF ABF180_12_R2 63.796
SF ABF180_12_R3 48.125
SF ABF180_13_R1 31.678
SF ABF180_13_R2 35.07
SF ABF180_13_R3 27.434
SF ABF180_14_R1 34.143
SF ABF180_14_R2 29.919
SF ABF180_14_R3 36.618
SF ABF180_15_R1 31.83
SF ABF180_15_R2 25.637
SF ABF180_15_R3 25.364
SF ABF

### Adding slack variables resulted in zero growth and all-zero fluxes
Adding slack variables to the infeasible subset introduces bias in flux prediction
### Switching to increasing the arbitrary scale for normalized global proteomics

In [150]:
# Find the minimum scaling factor
scaling_factor = 1.0
scaling_finished = False
while not scaling_finished:
    print('scaling_factor:', round(scaling_factor,1))
    scaling_finished = True
    for rep in global_prot.columns:
        if fluxes[rep].objective_value < 1e-6:
            with model:
                for k, v in const_dict.items():
                    model.reactions.get_by_id(k).lower_bound = normalized_line_rates[v][rep]
                eflux2_bounds = get_eflux2_bounds(model, scaling_factor*global_prot[rep])
                for r in model.reactions:
                    r.bounds = eflux2_bounds[r.id]
                sol = model.optimize()
            print(rep, sol.status, sol.objective_value)
            if sol.status is not 'optimal':
                scaling_factor = scaling_factor + 0.1
                scaling_finished = False
                break
    if scaling_finished:
        print('Done')

scaling_factor: 1.0
SF ABF180_5_R1 infeasible 0.06641065643871952
scaling_factor: 1.1
SF ABF180_5_R1 optimal 0.02845765303298798
SF ABF180_5_R2 optimal 0.025543955074796232
SF ABF180_5_R3 infeasible 0.0590199987540716
scaling_factor: 1.2
SF ABF180_5_R1 optimal 0.030846779830686375
SF ABF180_5_R2 optimal 0.027382101636454808
SF ABF180_5_R3 optimal 0.030665674412223263
SF ABF180_17_R1 optimal 0.019960575708049183
SF ABF180_17_R2 infeasible 0.025069975852005447
scaling_factor: 1.3
SF ABF180_5_R1 optimal 0.03309253544014601
SF ABF180_5_R2 optimal 0.028757827152223477
SF ABF180_5_R3 optimal 0.03301309262395901
SF ABF180_17_R1 optimal 0.021310984140991855
SF ABF180_17_R2 optimal 0.022452344863070232
SF ABF180_17_R3 optimal 0.029456387442109655
SF ABF180_19_R1 optimal 0.0325922886124592
SF ABF180_19_R3 optimal 0.025120411927803784
SF ABF180_20_R1 optimal 0.02523884267414161
SF ABF180_20_R2 optimal 0.0324823748637444
SF ABF180_21_R1 optimal 0.024893275096011316
SF ABF180_21_R2 optimal 0.031386

In [151]:
fluxes = {}
scaling_factor = 1.4
for rep in global_prot.columns:
    with model:
        for k, v in const_dict.items():
            model.reactions.get_by_id(k).lower_bound = normalized_line_rates[v][rep]
        try:
            print(rep)
            fluxes[rep] = EFlux2(model, scaling_factor*global_prot[rep])
        except TypeError:
            print(f"Replicate {rep} with glucose {glucose_uptake} and 3hp {secrete_3hp} is infeasible")

SF ABF180_1_R1
FBA status optimal
FBA solution 0.03639987726927395
EFlux2 status optimal
EFlux2 solution 24.6066916262007
SF ABF180_1_R2
FBA status optimal
FBA solution 0.035094373366872884
EFlux2 status optimal
EFlux2 solution 24.075557464919527
SF ABF180_1_R3
FBA status optimal
FBA solution 0.03435392579210334
EFlux2 status optimal
EFlux2 solution 23.76730215768786
SF ABF180_2_R1
FBA status optimal
FBA solution 0.039782992396607104
EFlux2 status optimal
EFlux2 solution 27.289556460628653
SF ABF180_2_R2
FBA status optimal
FBA solution 0.03897143754536338
EFlux2 status optimal
EFlux2 solution 26.588907587497424
SF ABF180_2_R3
FBA status optimal
FBA solution 0.041240446474831965
EFlux2 status optimal
EFlux2 solution 28.919636111194052
SF ABF180_3_R1
FBA status optimal
FBA solution 0.03595824842177282
EFlux2 status optimal
EFlux2 solution 26.88359153706103
SF ABF180_3_R2
FBA status optimal
FBA solution 0.03733771691804036
EFlux2 status optimal
EFlux2 solution 27.29167875962017
SF ABF180_

FBA status optimal
FBA solution 0.041639326290383756
EFlux2 status optimal
EFlux2 solution 29.679236324296312
SF ABF180_23_R2
FBA status optimal
FBA solution 0.037951355699593986
EFlux2 status optimal
EFlux2 solution 27.79436029431876
SF ABF180_23_R3
FBA status optimal
FBA solution 0.0403831900547402
EFlux2 status optimal
EFlux2 solution 31.87273822651893
SF ABF180_24_R1
FBA status optimal
FBA solution 0.032510949568529825
EFlux2 status optimal
EFlux2 solution 34.97585891032978
SF ABF180_24_R2
FBA status optimal
FBA solution 0.02814319625087355
EFlux2 status optimal
EFlux2 solution 25.364497790561163
SF ABF180_24_R3
FBA status optimal
FBA solution 0.03296931688508253
EFlux2 status optimal
EFlux2 solution 30.629011692821884


In [152]:
for rep in global_prot.columns:
    if fluxes[rep].status is not 'optimal':
        print(rep, fluxes[rep].status)

In [153]:
for rep in global_prot.columns:
    if fluxes[rep].objective_value < 1e-6:
        print(rep, fluxes[rep].status)

In [154]:
eflux_rates = pd.DataFrame(dict([(rep, fluxes[rep].to_frame()['fluxes']) for rep in fluxes]))
eflux_rates = eflux_rates*ref_line_glucose_rate
eflux_rates.head()

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
NE1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
NE2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
r1,0.450532,0.431722,0.421595,0.593783,0.583007,0.628937,0.595636,0.605161,0.572891,0.735088,...,0.704294,0.804294,0.784191,0.761066,0.646939,0.61222,0.709244,0.683248,0.528049,0.637687
r2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
r3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


## Process eflux_rates and reduce metabolic model

In [155]:
eflux_rates[abs(eflux_rates) < 1e-8] = 0.0

In [156]:
print('number of reactions', len(eflux_rates))
print('all zero flux', len(eflux_rates[(abs(eflux_rates) == 0).all(axis=1)]))
print('non-zero flux', len(eflux_rates[(abs(eflux_rates) != 0).any(axis=1)]))

number of reactions 2325
all zero flux 1709
non-zero flux 616


In [157]:
# remove reactions with all zero flux
reduced_model = model.copy()
zero_flux = eflux_rates.index[(abs(eflux_rates) == 0).all(axis=1)]
reduced_model.remove_reactions(zero_flux, remove_orphans=True)
eflux_rates.drop(zero_flux, inplace=True)

In [158]:
print('non-zero', len(eflux_rates))
print('non-negative', len(eflux_rates[(eflux_rates >= 0).all(axis=1)]))
print('non-positive', len(eflux_rates[(eflux_rates <= 0).all(axis=1)]))
print('mixed sign', len(eflux_rates[(eflux_rates > 0).any(axis=1) & (eflux_rates < 0).any(axis=1)]))

non-zero 616
non-negative 507
non-positive 70
mixed sign 39


In [159]:
# Update the lower bound of reactions with non-negative flux
for x in eflux_rates.index[(eflux_rates >= 0).all(axis=1)]:
    r = reduced_model.reactions.get_by_id(x)
    r.lower_bound = 0.0

In [160]:
# Reverse reactions with non-positive flux
for x in eflux_rates.index[(eflux_rates <= 0).all(axis=1)]:
    r = reduced_model.reactions.get_by_id(x)
    r.id = r.id + 'r'
    r.lower_bound = 0.0
    for m, s in r.metabolites.items():
        r.add_metabolites({m: -2*s})
    eflux_rates.loc[x] = -eflux_rates.loc[x]
    eflux_rates.rename(index={x: x+'r'}, inplace=True)

In [161]:
# Split reactions with mixed signs
for x in eflux_rates.index[(eflux_rates > 0).any(axis=1) & (eflux_rates < 0).any(axis=1)]:
    r = reduced_model.reactions.get_by_id(x)
    r.lower_bound = 0.0
    r_reverse = r.copy()
    r_reverse.id = r.id+'r'
    reduced_model.add_reactions([r_reverse])
    for m, s in r_reverse.metabolites.items():
        r_reverse.add_metabolites({m: -2*s})
    eflux_rates.loc[x+'r'] = -eflux_rates.loc[x]
    eflux_rates.loc[x][eflux_rates.loc[x] < 0] = 0.0
    eflux_rates.loc[x+'r'][eflux_rates.loc[x+'r'] < 0] = 0.0

In [162]:
for r in reduced_model.reactions:
    if r.lower_bound < 0.0:
        print(r)

In [163]:
print('reduced', len(eflux_rates))
print('non-negative', len(eflux_rates[(eflux_rates >= 0).all(axis=1)]))
print('non-positive', len(eflux_rates[(eflux_rates <= 0).all(axis=1)]))
print('mixed sign', len(eflux_rates[(eflux_rates > 0).any(axis=1) & (eflux_rates < 0).any(axis=1)]))

reduced 655
non-negative 655
non-positive 0
mixed sign 0


In [190]:
# Check reactions with zero flux in the reference strain
eflux_rates[eflux_rates[ref_line] == 0]

,SF ABF180_1_R1,SF ABF180_1_R2,SF ABF180_1_R3,SF ABF180_2_R1,SF ABF180_2_R2,SF ABF180_2_R3,SF ABF180_3_R1,SF ABF180_3_R2,SF ABF180_3_R3,SF ABF180_4_R1,...,SF ABF180_21_R3,SF ABF180_22_R1,SF ABF180_22_R2,SF ABF180_22_R3,SF ABF180_23_R1,SF ABF180_23_R2,SF ABF180_23_R3,SF ABF180_24_R1,SF ABF180_24_R2,SF ABF180_24_R3
r34,0.104603,0.102700,0.101043,0.215386,0.214261,0.183535,0.224811,0.232272,0.235707,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.222321,0.233287,0.195526,0.000000,0.188917,0.000000
r30,0.080211,0.088238,0.088851,0.129083,0.131313,0.119603,0.119651,0.136739,0.165179,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.119185,0.143053,0.022582,0.000000,0.089970,0.000000
r43,1.033248,1.031368,1.023933,0.761545,0.755434,0.745980,0.678120,0.706538,0.724512,0.205372,...,0.000000,0.000000,0.000000,0.000000,0.751873,0.723479,0.632142,0.216793,0.589583,0.493046
r59,2.691549,2.567341,2.505398,2.703495,2.607488,2.822241,1.942031,2.159732,2.022700,0.836451,...,0.000000,0.000000,0.000000,0.000000,2.859229,2.270301,2.384484,0.536749,0.865853,0.997074
r69ar,0.012839,0.002154,-0.000000,0.194621,0.191793,0.127964,0.242248,0.232163,0.208134,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,0.217660,0.224016,0.288232,-0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
r908r,0.170332,0.204228,0.222275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
r1152r,1.169316,1.140130,1.119059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
r1148r,0.758070,0.756774,0.751976,0.773613,0.771117,0.735873,0.803035,0.812086,0.819823,0.395936,...,0.097965,0.000000,0.000000,0.000000,0.791156,0.824376,0.765226,0.389751,0.791686,0.558670
r1133r,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,-0.000000,0.181618,...,0.330222,0.467316,0.501645,0.495576,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
# Either remove the reactions with zero flux in the reference strain
#ref_zero_flux = eflux_rates.index[eflux_rates[ref_line] == 0]
#eflux_rates.drop(ref_zero_flux, inplace=True)
#reduced_model.remove_reactions(ref_zero_flux, remove_orphans=True)

In [191]:
# or Add a small flux value in the reference strain
# use 1% of the non-zero minimum flux for each reaction
ref_zero_flux = eflux_rates[eflux_rates[ref_line] == 0]
non_zero_minimum = ref_zero_flux[ref_zero_flux > 0].min(axis=1)
eflux_rates[ref_line][eflux_rates[ref_line] == 0] = 0.01*non_zero_minimum

## Save eflux rates and reduced metabolic model

In [193]:
eflux_rates.to_csv('../../data/round2/Eflux2_flux_rates.csv')

In [194]:
cobra.io.save_json_model(reduced_model, '../../models/iJB1325_HP.nonnative_genes.pubchem.flipped.nonzero.reduced.round2.json')